In [2]:
# Importation des bibliothèques nécessaires
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import os

In [3]:
# Initialisation de SparkSession
spark = SparkSession.builder \
    .appName("WildLens ETL") \
    .getOrCreate()

/home/jupyter/.local/lib/python3.9/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/13 10:48:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Chargement des informations sur les espèces
species_info_path = './data/csv/infos_especes.csv'
species_info_df = spark.read.csv(species_info_path, header=True, inferSchema=True)
species_info_df.show(5)


+-------------------------------------------------------------------+
|Espèce;Description;Nom latin;Famille;Taille;Région;Habitat;Fun fact|
+-------------------------------------------------------------------+
|                                               Castor;Le castor ...|
|                                               Chat;Le Chat sauv...|
|                                               Chien;Alors qu'on...|
|                                                   Coyote;Le coyote|
|                                               Ecureuil;L’écureu...|
+-------------------------------------------------------------------+
only showing top 5 rows



In [1]:
import sys
print("sys.executable")

sys.executable
